In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/multi-lingual-sentiment-analysis/sample_submission.csv
/kaggle/input/multi-lingual-sentiment-analysis/train.csv
/kaggle/input/multi-lingual-sentiment-analysis/test.csv
/kaggle/input/llama-3.1/transformers/8b-instruct/2/model.safetensors.index.json
/kaggle/input/llama-3.1/transformers/8b-instruct/2/model-00003-of-00004.safetensors
/kaggle/input/llama-3.1/transformers/8b-instruct/2/config.json
/kaggle/input/llama-3.1/transformers/8b-instruct/2/LICENSE
/kaggle/input/llama-3.1/transformers/8b-instruct/2/model-00001-of-00004.safetensors
/kaggle/input/llama-3.1/transformers/8b-instruct/2/README.md
/kaggle/input/llama-3.1/transformers/8b-instruct/2/USE_POLICY.md
/kaggle/input/llama-3.1/transformers/8b-instruct/2/tokenizer.json
/kaggle/input/llama-3.1/transformers/8b-instruct/2/tokenizer_config.json
/kaggle/input/llama-3.1/transformers/8b-instruct/2/model-00004-of-00004.safetensors
/kaggle/input/llama-3.1/transformers/8b-instruct/2/special_tokens_map.json
/kaggle/input/llama-3.1/

In [2]:
%%capture
!pip install bitsandbytes
!pip install unsloth
!pip install accelerate
!pip install peft
!pip install torch==2.1.2 --force-reinstall
!pip install --upgrade transformers
!pip install --upgrade unsloth

In [3]:
import torch._dynamo
print(torch.__version__)

2.6.0+cu124


In [4]:
from datetime import datetime
from unsloth import FastLanguageModel
import torch
import pandas as pd
import transformers
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from peft import prepare_model_for_kbit_training, LoraConfig, PeftModel, get_peft_model
from datasets import load_dataset, Dataset
from sklearn.model_selection import train_test_split
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [5]:
file_path = "/kaggle/input/multi-lingual-sentiment-analysis/train.csv"
test_file_path = "/kaggle/input/multi-lingual-sentiment-analysis/test.csv"
model_path = "/kaggle/input/llama-3.1/transformers/8b-instruct/2"
max_seq_length = 4096 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.
map_dict = {
    "as": "Assamese",
    "bd": "Bodo",
    "bn": "Bengali",
    "gu": "Gujarati",
    "hi": "Hindi",
    "kn": "Kannada",
    "ml": "Malayalam",
    "mr": "Marathi",
    "or": "Odia",
    "pa": "Punjabi",
    "ta": "Tamil",
    "te": "Telugu",
    "ur": "Urdu"
}

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = model_path,
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

==((====))==  Unsloth 2025.2.12: Fast Llama patching. Transformers: 4.48.3.
   \\   /|    GPU: Tesla T4. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

/kaggle/input/llama-3.1/transformers/8b-instruct/2 does not have a padding token! Will use pad_token = <|finetune_right_pad_id|>.


In [6]:
# Adding LoRA adapters
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, 
    bias = "none",    
    use_gradient_checkpointing = "unsloth", 
    random_state = 3407,
    use_rslora = False,  
    loftq_config = None, 
)

Unsloth 2025.2.12 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [7]:
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Predict the sentiment of input as 1 (positive) or 0 (negative), output 0 or 1. This sentence comes from the {} language.

### Input:
{}

### Response:
{}"""

EOS_TOKEN = tokenizer.eos_token 
def formatting_prompts_func(examples):
    texts = []
    new_labels = []
    sentences = examples['sentence']
    langs = examples['language']
    labels = examples['label']
    for sent, lang, lab in zip(sentences, langs, labels):
        new_lab = 1 if lab == 'Positive' else 0
        text = alpaca_prompt.format(map_dict[lang], sent, new_lab) + EOS_TOKEN
        texts.append(text)
        new_labels.append(new_lab)
    return { "text" : texts, "label": new_labels}

dataset = load_dataset("csv", data_files = [file_path])["train"]
dataset = dataset.map(formatting_prompts_func, batched = True,)
split_dataset = dataset.train_test_split(test_size = 0.2, seed = 62)
train_dataset, valid_dataset = split_dataset['train'], split_dataset['test']
train_dataset, valid_dataset

Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

(Dataset({
     features: ['ID', 'sentence', 'label', 'language', 'text'],
     num_rows: 800
 }),
 Dataset({
     features: ['ID', 'sentence', 'label', 'language', 'text'],
     num_rows: 200
 }))

In [8]:
train_dataset[0], valid_dataset[0]

({'ID': 150,
  'sentence': 'ସାମ୍ପ୍ରତିକ ସପ୍ତାହଗୁଡିକରେ, ସୋନିଲିଭ୍ କିଛି ବାସ୍ତବରେ ଲୋକପ୍ରିୟ ସିରିଜ୍ ଏବଂ ଲଗାତାର ଦେଖିବା ପାଇଁ କିଛି ଭଲ ବିଷୟବସ୍ତୁ ତାଲିକାଭୁକ୍ତ କରୁଛି |',
  'label': 1,
  'language': 'or',
  'text': 'Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\nPredict the sentiment of input as 1 (positive) or 0 (negative), output 0 or 1. This sentence comes from the Odia language.\n\n### Input:\nସାମ୍ପ୍ରତିକ ସପ୍ତାହଗୁଡିକରେ, ସୋନିଲିଭ୍ କିଛି ବାସ୍ତବରେ ଲୋକପ୍ରିୟ ସିରିଜ୍ ଏବଂ ଲଗାତାର ଦେଖିବା ପାଇଁ କିଛି ଭଲ ବିଷୟବସ୍ତୁ ତାଲିକାଭୁକ୍ତ କରୁଛି |\n\n### Response:\n1<|eot_id|>'},
 {'ID': 466,
  'sentence': 'क्रोमा एसी का एआई मोड, जिसे ऑटो ऑपरेशन मोड के रूप में भी जाना जाता है, कमरे के तापमान के आधार पर पंखे की गति और तापमान को ऑटोमेटिकली सेट करता है। चूंकि यह आर्टिफीशियल इंटेलिजेंस पर काम करता है, इसलिए परिणाम इतने अच्छे नहीं हैं।',
  'label': 0,
  'language': 'hi',
  'text': 'Below is an instruct

In [9]:
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = train_dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, 
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 8,
        warmup_steps = 5,
        num_train_epochs = 5,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 10,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "",
        report_to = "none", 
    ),
)

Applying chat template to train dataset (num_proc=2):   0%|          | 0/800 [00:00<?, ? examples/s]

Tokenizing train dataset (num_proc=2):   0%|          | 0/800 [00:00<?, ? examples/s]

Tokenizing train dataset (num_proc=2):   0%|          | 0/800 [00:00<?, ? examples/s]

In [10]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 800 | Num Epochs = 5
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 8
\        /    Total batch size = 16 | Total steps = 250
 "-____-"     Number of trainable parameters = 41,943,040


Step,Training Loss
10,1.336300
20,0.709700
30,0.680200
40,0.702200
50,0.601200
60,0.552600
70,0.577200
80,0.568800
90,0.583000
100,0.543600


In [12]:
FastLanguageModel.for_inference(model) 
example = train_dataset[0]
inputs = tokenizer(
[
    alpaca_prompt.format(map_dict[example['language']], example['sentence'], '')
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 64, use_cache = True, temperature=0.1)
result = tokenizer.batch_decode(outputs)[0].split('Response')[-1].split('\n')[1][0]
result, example['label']

('1', 1)

In [13]:
FastLanguageModel.for_inference(model)
def get_results(examples):
    results = []
    sentences = examples['sentence']
    langs = examples['language']
    inputs = tokenizer(
        [
            alpaca_prompt.format(map_dict[langs], sentences, '')
        ], return_tensors = "pt").to("cuda")
    outputs = model.generate(**inputs, max_new_tokens = 64, use_cache = True)
    result = int(tokenizer.batch_decode(outputs)[0].split('Response')[-1].split('\n')[1][0])
    return {'result': result}

train_dataset = train_dataset.map(get_results)
valid_dataset = valid_dataset.map(get_results)

Parameter 'function'=<function get_results at 0x79a8ae6f39a0> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


Map:   0%|          | 0/800 [00:00<?, ? examples/s]

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

In [14]:
def calculate_F1(actual, predicted):
    tp, tn, fp, fn = 0, 0, 0, 0
    for i in range(len(actual)):
        if actual[i] == predicted[i]:
            if actual[i] == 1: tp += 1
            else: tn += 1
        else:
            if actual[i] == 1: fn += 1
            else: fp += 1
    precision = tp / (tp+fp)
    recall = tp / (tp+fn)
    accuracy = (tp+tn) / (tp+tn+fp+fn)
    f1_score = (2 * precision * recall) / (precision+recall)
    print(f'Accuracy: {accuracy}, Precision: {precision}, Recall: {recall}')
    print(f'F1 Score: {f1_score}')

In [15]:
print('Train Metrics')
calculate_F1(train_dataset[:]['label'], train_dataset[:]['result'])
print('Validation Metrics')
calculate_F1(valid_dataset[:]['label'], valid_dataset[:]['result'])

Train Metrics
Accuracy: 0.99, Precision: 0.9927536231884058, Recall: 0.9879807692307693
F1 Score: 0.9903614457831326
Validation Metrics
Accuracy: 0.94, Precision: 0.9247311827956989, Recall: 0.945054945054945
F1 Score: 0.9347826086956522


In [16]:
model.save_pretrained("lora_model") # Local saving
tokenizer.save_pretrained("lora_model")

('lora_model/tokenizer_config.json',
 'lora_model/special_tokens_map.json',
 'lora_model/tokenizer.json')

In [17]:
test_dataset = load_dataset("csv", data_files = [test_file_path])["train"]
test_dataset = test_dataset.map(get_results)

Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

In [18]:
predictions = test_dataset[:]['result']
predictions = ['Positive' if predictions[i] == 1 else 'Negative' for i in range(len(predictions))]
len(predictions)

100

In [21]:
dl = pd.DataFrame({'ID': list(range(1,len(predictions)+1)), 'Label': predictions})
dl.to_csv('/kaggle/working/submission.csv', index=False)